# Modelagem
Neste notebook iremos desenvolver, testar e validas os modelos de recomendação musical, nesta MVP nossa maior preocupação não é a acurácia e sim o bom entendimento dos modelos e como melhorar-los.
Para tanto testaremos três técnicas:
* Popularidade
* Similaridade
* Filtragem colaborativa com SVD

Utilizaremos como direcional NSM (North Star Metric) uma relação de "peso" para a música entre a quantidade de vezes que aquele usuario ouviu e as vezes ouvidas no geral.



In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [73]:
df = pd.read_parquet(r'data/processed_recommendation_data.parquet')

 ## Criando nossa north metric star

Nort star metric: definição de uma métrica de sucesso ou não para se guiar

vezes que o usuario ouviu / quantidade de vezes ouvidas

In [349]:
df['count_plays'] = df.groupby('id_tracks')['plays'].sum().reset_index()['plays']

In [350]:
df['count_plays'] = df['count_plays'].fillna(1)

In [351]:
C = df['plays'].mean()
m = df['count_plays'].quantile(0.95)

In [352]:
def weighted_rating(x):
    v = x['plays']
    R = x['count_plays']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [353]:
df['wr'] = df.apply(weighted_rating, axis=1)

In [354]:
df = df.sort_values('wr', ascending=False)
df

,id_date,user_id,id_tracks,plays,holiday,id_artist,id_genre,Feature1,Feature2,Feature3,...,Feature7,Feature8,Feature9,Feature10,acima_media_track,abaixo_media_track,acima_media_artist,abaixo_media_artist,count_plays,wr
560,2023-02-02,47,3622,19,1,493.0,4.0,419.0,927.0,568.0,...,1185.0,755.0,345.0,1491.0,1,0,1,0,55.0,26.778372
400,2023-01-23,11,1508,9,1,318.0,4.0,450.0,1744.0,294.0,...,1198.0,393.0,374.0,1449.0,0,1,0,1,80.0,25.382853
488,2023-01-29,42,1099,19,1,190.0,3.0,442.0,1331.0,888.0,...,1166.0,517.0,458.0,1239.0,1,0,1,0,48.0,24.170529
1021,2023-03-01,32,3409,19,1,172.0,3.0,385.0,1163.0,2206.0,...,1217.0,680.0,577.0,1659.0,1,0,1,0,46.0,23.425431
1900,2023-04-20,19,326,19,1,320.0,3.0,471.0,1494.0,676.0,...,1193.0,612.0,836.0,473.0,1,0,1,0,46.0,23.425431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3536,2023-07-21,5,3659,19,1,230.0,1.0,425.0,1396.0,1111.0,...,1279.0,633.0,434.0,-641.0,1,0,1,0,1.0,6.660725
2747,2023-06-05,34,1735,19,1,357.0,3.0,415.0,1495.0,292.0,...,1111.0,361.0,592.0,1283.0,1,0,1,0,1.0,6.660725
3066,2023-06-23,38,3997,19,1,293.0,4.0,452.0,1077.0,367.0,...,1289.0,574.0,459.0,901.0,1,0,1,0,1.0,6.660725
2758,2023-06-06,3,1418,19,1,436.0,4.0,400.0,1348.0,-77.0,...,1291.0,558.0,-147.0,1277.0,1,0,1,0,1.0,6.660725


## Content Based Recommender




In [356]:
interactions_train_df, interactions_test_df = train_test_split(df,
                                   stratify=df['user_id'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 2975
# interactions on Test set: 744


## Modelo de Popularidade

In [ ]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [ ]:
#Computes the most popular items
item_popularity_df = df.groupby('id_tracks')['wr'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,id_tracks,wr
0,3230,59.000062
1,1709,56.060497
2,1891,55.304937
3,939,52.239920
4,293,52.213197
5,3600,52.060000
6,3790,51.692239
7,3622,51.356487
8,1804,49.735916
9,1057,49.619433


In [363]:

class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df.sort_values('wr', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'id_tracks', 
                                                          right_on = 'id_tracks')[['wr', 'id_tracks']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, df)

In [656]:
df['user_id']

560     47
400     11
488     42
1021    32
1900    19
        ..
3536     5
2747    34
3066    38
2758     3
3530     2
Name: user_id, Length: 3719, dtype: int32

In [364]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['id_tracks']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [367]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(df['id_tracks'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['id_tracks']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['id_tracks'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['id_tracks'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['id_tracks'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['id_tracks'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
    
model_evaluator = ModelEvaluator()    

In [368]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
48 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.11036339165545088, 'recall@10': 0.2072678331090175}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
17,2,5,24,0.083333,0.208333,1
7,6,9,22,0.272727,0.409091,30
37,1,3,21,0.047619,0.142857,15
28,2,6,19,0.105263,0.315789,42
15,2,4,19,0.105263,0.210526,39
22,0,1,19,0.000000,0.052632,9
11,4,5,19,0.210526,0.263158,16
10,1,1,18,0.055556,0.055556,22
13,2,4,18,0.111111,0.222222,13
38,3,5,18,0.166667,0.277778,6


In [663]:
predict = df.groupby(['user_id']).apply(lambda grp: popularity_model.recommend_items(grp).head(2)).reset_index()
predict.columns = ['user_id', 'top1_2', 'id_tracks', 'wr']

In [665]:
predict.to_parquet(r'data/predict_recommendation_data.parquet')

## Conclusão popularidade

## Similarity method Cosine

In [369]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
df_student = df.drop('id_date', axis = 1).set_index(['user_id', 'id_tracks'])


In [370]:
sim = pd.DataFrame(cosine_similarity(df_student), 
                  index=df_student.index, columns=df_student.index).reset_index()



In [371]:
sim = sim.median().reset_index()

In [372]:
sim[sim.id_tracks != ''].sort_values(by = 'user_id')

,user_id,id_tracks,0
255,1,668,0.914148
1851,1,2816,0.881484
2993,1,1511,0.851479
2999,1,3921,0.847409
1797,1,3940,0.882817
...,...,...,...
1786,49,480,0.872431
1744,49,2636,0.819510
1653,49,2093,0.914241
3512,49,859,0.839908


In [373]:
df_final = df[['id_tracks', 'user_id', 'wr']].merge(sim)

In [374]:
df_final.sort_values(by = ['id_tracks', 0], ascending=False)#.groupby(['user_id','id_tracks'])[['wr', 0]].head(3)

,id_tracks,user_id,wr,0
1240,3999,38,10.708101,0.863237
3802,3997,38,6.660725,0.892787
3195,3995,39,7.713883,0.881146
3196,3994,11,7.704377,0.865533
746,3994,13,11.817424,0.865528
...,...,...,...,...
209,8,49,15.229352,0.914216
842,7,6,11.553584,0.825476
1267,7,9,10.681187,0.825468
2532,4,9,8.802621,0.772160


## Conclusão similaridade

Collaborative Filtering mode

In [391]:
summarize_data_train = interactions_train_df.drop('id_date', axis = 1).groupby(['user_id', 'id_tracks'])['wr'].mean().reset_index()

In [392]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = summarize_data_train.pivot(index='user_id', 
                                                          columns='id_tracks', 
                                                          values='wr').fillna(0)

users_items_pivot_matrix_df.head(10)

id_tracks,4,7,8,10,11,13,15,16,18,20,...,3982,3983,3984,3985,3987,3988,3989,3990,3991,3994
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,...,8.041389,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,...,0.000000,9.681187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,9.01936,0.0,0.0,11.11456,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,8.217424,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,8.802621,10.681187,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [393]:
minimum_rating = min(df['wr'].values)
maximum_rating = max(df['wr'].values)
print(minimum_rating,maximum_rating)

6.660724736251048 26.778371795074577


In [378]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix_df)
users_items_pivot_sparse_matrix

<49x2115 sparse matrix of type '<class 'numpy.float64'>'
	with 2946 stored elements in Compressed Sparse Row format>

In [379]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)

In [380]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [381]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[-0.11966872, -0.14520721, -0.1355284 , ...,  0.01946106,
        -0.16087609,  0.08473167],
       [ 0.4782726 ,  0.58034069, -0.3901191 , ..., -0.25755618,
        -0.29814852, -0.43997115],
       [ 1.12792316,  1.3686331 ,  0.6034308 , ..., -0.09296673,
         0.47631288,  0.3284674 ],
       ...,
       [ 0.27142898,  0.32935461,  0.69974883, ..., -0.04308368,
        -0.01956869,  0.74176857],
       [ 0.05363011,  0.06507531,  0.27448581, ...,  0.09969796,
        -0.01050831,  0.48347668],
       [ 0.09349404,  0.11344659,  4.66544535, ...,  1.94581895,
         0.33080002,  1.67199461]])

In [382]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

In [383]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,1,2,3,4,5,6,7,8,9,10,...,40,41,42,43,44,45,46,47,48,49
id_tracks,,,,,,,,,,,,,,,,,,,,,
4,0.170821,0.192102,0.215223,0.182187,0.181502,0.165694,0.191459,0.198718,0.447228,0.169834,...,0.187026,0.175702,0.176428,0.173879,0.190494,0.184695,0.163156,0.184740,0.176989,0.178408
7,0.169912,0.195734,0.223790,0.183703,0.182872,0.163691,0.194954,0.203762,0.505307,0.168714,...,0.189575,0.175835,0.176716,0.173622,0.193784,0.186747,0.160611,0.186802,0.177396,0.179118
8,0.170257,0.161196,0.196556,0.206626,0.143611,0.222730,0.190508,0.198511,0.190243,0.210240,...,0.183892,0.285716,0.137564,0.191219,0.186767,0.225688,0.183879,0.199984,0.184849,0.341123
10,0.173740,0.180716,0.176087,0.176256,0.175467,0.172835,0.175787,0.176102,0.177001,0.176763,...,0.175249,0.183560,0.173754,0.176332,0.176145,0.173818,0.175691,0.176653,0.176123,0.181004
11,0.171195,0.195305,0.178141,0.178432,0.171771,0.184202,0.177460,0.181675,0.172721,0.179257,...,0.176149,0.181930,0.196304,0.179294,0.176983,0.201021,0.180201,0.180759,0.175187,0.195575
13,0.157672,0.170817,0.223239,0.189300,0.171021,0.162757,0.175285,0.180140,0.193375,0.176425,...,0.179800,0.160805,0.170621,0.169888,0.164877,0.178719,0.158459,0.190954,0.179447,0.185141
15,0.166197,0.197589,0.250272,0.197651,0.236762,0.212733,0.182021,0.186126,0.193201,0.255867,...,0.194602,0.188388,0.213623,0.192432,0.165093,0.220783,0.198969,0.181128,0.167863,0.240009
16,0.178589,0.232848,0.169661,0.176857,0.201097,0.170018,0.175126,0.177855,0.160594,0.171960,...,0.173763,0.179976,0.185034,0.179688,0.179496,0.195917,0.183790,0.188096,0.178558,0.169320
18,0.164810,0.186334,0.204261,0.179200,0.166986,0.173411,0.179501,0.180038,0.224773,0.195133,...,0.179601,0.191291,0.165402,0.182636,0.181721,0.171613,0.174697,0.180641,0.175996,0.195334


In [388]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'wr'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions.sort_values('wr', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'id_tracks', 
                                                          right_on = 'id_tracks')[['wr', 'id_tracks']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(cf_preds_df, df)

In [389]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
48 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.03768506056527591, 'recall@10': 0.06594885598923284}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
17,1,2,24,0.041667,0.083333,1
7,2,2,22,0.090909,0.090909,30
37,2,3,21,0.095238,0.142857,15
28,0,1,19,0.000000,0.052632,42
15,0,1,19,0.000000,0.052632,39
22,1,1,19,0.052632,0.052632,9
11,0,1,19,0.000000,0.052632,16
10,0,0,18,0.000000,0.000000,22
13,1,1,18,0.055556,0.055556,13
38,0,0,18,0.000000,0.000000,6


## Conclusão

Neste notebook testamos três diferentes formas de recomendação, das quais duas foram baseadas no conteudo e uma baseada em filtragem colaborativa, o modelo que melhor desempenhou foi  